- 데이터 불러오기
- 모델
- 최적화
- 트레이닝
- 이벨류에이션

# TensorFlow 2.0

In [5]:
import tensorflow as tf
from tensorflow.keras import layers #모델 구현

from tensorflow.keras import datasets #데이터 예제

## Hyperparameter Tunning

In [6]:
num_epochs = 1
batch_size = 64

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28, 28, 1)
num_classes = 10

## Preprocess

In [8]:
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()
#데이터 불러옴

In [9]:
#차원늘리기
train_x = train_x[..., tf.newaxis]
test_x = test_x[..., tf.newaxis]
#Normalization/rescale
train_x = train_x / 255.
test_x = test_x / 255.

## Build Model

In [10]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [11]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [12]:
#학습
model.fit(train_x, train_y,   
             batch_size=batch_size, 
             shuffle=True)

model.evaluate(test_x, test_y, batch_size=batch_size)

157/157 [==============================] - 6s 35ms/step - loss: 0.0598 - accuracy: 0.9809


[0.05979456752538681, 0.98089998960495]

# PyTorch

In [13]:
import torch 

import torch.nn as nn #모델 설계
import torch.nn.functional as F #모델 설계
import torch.optim as optim #컴파일할때 필요

from torchvision import datasets, transforms #이미지 비젼 다루기 위한 라이브러리
#데이터 로더를 만들어야함 (텐서플로우와 다르게)

In [14]:
seed = 1 #파이토치 모델이 학습마다 랜덤하게 학습하기 때문에 랜덤값 고정하기 위해 seed

lr = 0.001 #learning rate
momentum = 0.5

batch_size = 64 
test_batch_size = 64

epochs = 5

no_cuda = False #쿠다가 설치가 안되었기 때문에
log_interval = 100 #100번 스텝 돌 때마다 로그를 보여줌
#텐서플로우는 프로그레스 바가 자동으로 생김

## Model

In [16]:
class Net(nn.Module):
    def __init__(self): #파라메터가 필요한
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)#텐서와 다르게 인채널도 필요
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500) #텐서의 덴스
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Preprocess

In [18]:
torch.manual_seed(seed)#랜덤을 하더라도 똑같은 방법으로 생성하게

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
#cuda = ndivia에서 제공하는 그래픽카드(GPU)
#딥러닝은 계산해야 할 양이 많기 때문에 그래픽카드를 활용하기도 한다.

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)



test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

## Optimization

In [19]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

## Training

In [ ]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        #여기까지 1 epoch
    
    # Test mode
    model.eval()  # batch norm이나 dropout 등을 train mode 변환
    test_loss = 0
    correct = 0
    with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # pred와 target과 같은지 확인
            #정확도 계산
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
#텐서는 fit evaluate으로 간단하게 학습시킴
#파이토치는 직접 기울기 계산후 업데이트하고 로그를 직접 설정하고,,

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299172
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.233125
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.125046
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.954126
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.746479
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.243096
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.026327
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.617802
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.717742
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.411956

Test set: Average loss: 0.4822, Accuracy: 8670/10000 (87%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.718501
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.433117
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.361750
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.419109
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.372064
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.486471
